In [1]:
class Piece:
      
    def __init__(self, position='A1', color='white'):
        self.color = color
        self.position = position.upper()
        self.status = 'man'    
    
    def status_change(self):
        self.status = 'king'
        
    def __str__(self):
        if self.status == 'man':
            return '○' if self.color == 'white' else '●'
        return self.color[0].lower()

In [2]:
class CheckerBoard:
    
    @staticmethod
    def indxs2pos(indxs=(0, 0)):
        return f'{chr(indxs[1] + ord("A"))}{indxs[0] + 1}'
    
    @staticmethod
    def position_change(old_position='A1', position='A1'):
        h = ord(position[0].upper()) - ord(old_position[0].upper())
        v = ord(position[1]) - ord(old_position[1])
        return (h, v)
    
    def __init__(self):
        self.paint = [[1, 0, 1, 0, 1, 0, 1, 0], 
                      [0, 1, 0, 1, 0, 1, 0, 1], 
                      [1, 0, 1, 0, 1, 0, 1, 0], 
                      [0, 1, 0, 1, 0, 1, 0, 1], 
                      [1, 0, 1, 0, 1, 0, 1, 0], 
                      [0, 1, 0, 1, 0, 1, 0, 1], 
                      [1, 0, 1, 0, 1, 0, 1, 0], 
                      [0, 1, 0, 1, 0, 1, 0, 1]]
        
    def __getitem__(self, indxs):
        if isinstance(indxs, str):
            pos = indxs
            indxs = (int(pos[1]) - 1, ord(pos[0].upper()) - ord('A'))
        return self.paint[indxs[0]][indxs[1]]
    
    def __setitem__(self, indxs, value):
        if isinstance(indxs, str):
            pos = indxs
            indxs = (int(pos[1]) - 1, ord(pos[0].upper()) - ord('A'))
        self.paint[indxs[0]][indxs[1]] = value
        
    def __iter__(self):
        for j in range(8):
            yield from [(CheckerBoard.indxs2pos((j, i)), self.paint[j][i]) for i in range(8) if self.paint[j][i] != 0]
        
    def can_move_or_capture(self, piece, new_position='A1'):
        new_position = new_position.upper()
        empty_fields = list(pos for pos, value in self.__iter__() if value == 1)
        if new_position in empty_fields:
            h, v = CheckerBoard.position_change(piece.position, new_position)
            if piece.status == 'man':
                if abs(h) == 1 and v == (1 if piece.color == 'white' else -1) * 1:
                    return True
                if abs(h) == 2 and abs(v) == 2:
                    other_piece_position = f'{chr(ord(piece.position[0]) + h // 2)}{chr(ord(piece.position[1]) + v // 2)}'
                    if isinstance(self[other_piece_position], Piece):
                        if self[other_piece_position].color != piece.color:
                            return other_piece_position
            else:
                if abs(h) == abs(v):
                    res = []
                    for i in range(1, abs(h) + 1):
                        step = f'{chr(ord(piece.position[0]) + h // abs(h) * i)}{chr(ord(piece.position[1]) + v // abs(v) * i)}'           
                        if isinstance(self[step], Piece):
                            print(step)
                            if self[step].color != piece.color:
                                res.append(step)
                            else:
                                return False
                    return tuple(res)
                    
        return False
    
    def put_piece(self, old_position=None, new_position=None, new_piece=None):
        if isinstance(new_piece, Piece):
            pos = new_piece.position
            self.paint[int(pos[1]) - 1][ord(pos[0].upper()) - ord('A')] = new_piece
            return None
        piece = self[old_position]
        if isinstance(piece, Piece):
            if not new_position is None:
                move = self.can_move_or_capture(piece, new_position)
                if isinstance(move, tuple):
                    pos = piece.position
                    piece.position = new_position
                    self[new_position] = piece
                    for capture in move:
                        self[capture] = 1
                    self[pos] = 1
                elif isinstance(move, str):
                    pos = piece.position
                    piece.position = new_position
                    self[new_position] = piece
                    self[move] = 1
                    self[pos] = 1
                elif move:
                    pos = piece.position
                    piece.position = new_position
                    self[new_position] = piece
                    self[pos] = 1
                else:
                    print('WRONG  MOVE!!!')
                    return None
                
                if (new_position[1], piece.color) == ('1', 'black') or (new_position[1], piece.color) == ('8', 'white'):
                    piece.status_change()
                
        else:
            print('WRONG  MOVE!!!')
        
    def print_board(self):
        for i in reversed(range(8)):
            line = []
            for q in self.paint[i]:
                if q == 1:
                    line.append('▪')
                elif q == 0:
                    line.append('▫')
                else:
                    line.append(q)
            print((i + 1), ' ', *line)
        print()
        print(*list('  ABCDEFGH'))
        
    

In [3]:
class Game:
    
    def __init__(self):
        game_board = CheckerBoard()
        pieces = list(iter(game_board))
        pieces_num = len(pieces)
        for q in range(12):
            white_piece = Piece(pieces[q][0])
            game_board.put_piece(new_piece=white_piece)
        for q in range(12):
            black_piece = Piece(pieces[pieces_num - q - 1][0], 'black')
            game_board.put_piece(new_piece=black_piece)
        self.game_board = game_board
        
        
    

In [4]:
g = Game()
g.game_board.print_board()

8   ▫ ● ▫ ● ▫ ● ▫ ●
7   ● ▫ ● ▫ ● ▫ ● ▫
6   ▫ ● ▫ ● ▫ ● ▫ ●
5   ▪ ▫ ▪ ▫ ▪ ▫ ▪ ▫
4   ▫ ▪ ▫ ▪ ▫ ▪ ▫ ▪
3   ○ ▫ ○ ▫ ○ ▫ ○ ▫
2   ▫ ○ ▫ ○ ▫ ○ ▫ ○
1   ○ ▫ ○ ▫ ○ ▫ ○ ▫

    A B C D E F G H


In [5]:
class TestGame(Game):
    
    def start(self):
        while True:
            line = input()
            if line == 'stop':
                break
            old, new = line.split()
            self.game_board.put_piece(old, new)
            self.game_board.print_board()

In [6]:
test = TestGame()
test.start()

b6 a5
8   ▫ ● ▫ ● ▫ ● ▫ ●
7   ● ▫ ● ▫ ● ▫ ● ▫
6   ▫ ▪ ▫ ● ▫ ● ▫ ●
5   ● ▫ ▪ ▫ ▪ ▫ ▪ ▫
4   ▫ ▪ ▫ ▪ ▫ ▪ ▫ ▪
3   ○ ▫ ○ ▫ ○ ▫ ○ ▫
2   ▫ ○ ▫ ○ ▫ ○ ▫ ○
1   ○ ▫ ○ ▫ ○ ▫ ○ ▫

    A B C D E F G H
c7 b6
8   ▫ ● ▫ ● ▫ ● ▫ ●
7   ● ▫ ▪ ▫ ● ▫ ● ▫
6   ▫ ● ▫ ● ▫ ● ▫ ●
5   ● ▫ ▪ ▫ ▪ ▫ ▪ ▫
4   ▫ ▪ ▫ ▪ ▫ ▪ ▫ ▪
3   ○ ▫ ○ ▫ ○ ▫ ○ ▫
2   ▫ ○ ▫ ○ ▫ ○ ▫ ○
1   ○ ▫ ○ ▫ ○ ▫ ○ ▫

    A B C D E F G H
d6 c5
8   ▫ ● ▫ ● ▫ ● ▫ ●
7   ● ▫ ▪ ▫ ● ▫ ● ▫
6   ▫ ● ▫ ▪ ▫ ● ▫ ●
5   ● ▫ ● ▫ ▪ ▫ ▪ ▫
4   ▫ ▪ ▫ ▪ ▫ ▪ ▫ ▪
3   ○ ▫ ○ ▫ ○ ▫ ○ ▫
2   ▫ ○ ▫ ○ ▫ ○ ▫ ○
1   ○ ▫ ○ ▫ ○ ▫ ○ ▫

    A B C D E F G H
g3 f4
8   ▫ ● ▫ ● ▫ ● ▫ ●
7   ● ▫ ▪ ▫ ● ▫ ● ▫
6   ▫ ● ▫ ▪ ▫ ● ▫ ●
5   ● ▫ ● ▫ ▪ ▫ ▪ ▫
4   ▫ ▪ ▫ ▪ ▫ ○ ▫ ▪
3   ○ ▫ ○ ▫ ○ ▫ ▪ ▫
2   ▫ ○ ▫ ○ ▫ ○ ▫ ○
1   ○ ▫ ○ ▫ ○ ▫ ○ ▫

    A B C D E F G H
f6 e5
8   ▫ ● ▫ ● ▫ ● ▫ ●
7   ● ▫ ▪ ▫ ● ▫ ● ▫
6   ▫ ● ▫ ▪ ▫ ▪ ▫ ●
5   ● ▫ ● ▫ ● ▫ ▪ ▫
4   ▫ ▪ ▫ ▪ ▫ ○ ▫ ▪
3   ○ ▫ ○ ▫ ○ ▫ ▪ ▫
2   ▫ ○ ▫ ○ ▫ ○ ▫ ○
1   ○ ▫ ○ ▫ ○ ▫ ○ ▫

    A B C D E F G H
b8 c7
8   ▫ ▪ ▫ ● ▫ ● ▫ ●
7   ● ▫ ● ▫ ● ▫ ● ▫
6   ▫ ● ▫ ▪ ▫ ▪ ▫ ●